In [36]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("/content/IMDB Dataset.csv")
df=df.iloc[:5000,]

df.shape

(5000, 2)

In [4]:
df["sentiment"].value_counts()
print(df["review"][0])
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df.duplicated().sum()

df.isnull().sum()
df.info()

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import re
from nltk.corpus import stopwords
def preprocessing_data(text):

    text =text.lower() #convert the text into the lower case

    tags = re.compile("<.*?>")
    text=re.sub(tags," ",text) #to remove htmls tags.

    #now remove urls
    urls = re.compile(r'https?://\S+|www\.\S+')
    text = re.sub(urls," ",text)

    #removing_stopwords
    new_text=[]
    for w in text.split():
        if w in stopwords.words("english"):
            new_text.append("")
        else:
            new_text.append(w)
    return " ".join(new_text)


In [7]:
df["review"]=df["review"].apply(preprocessing_data)


In [8]:
df.head()
df.shape

(4997, 2)

In [9]:
#now labelencoder to final_columns

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df["sentiment"]=encoder.fit_transform(df["sentiment"])

df.head(4)
df.shape

(4997, 2)

In [10]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1:]
X.shape,y.shape

((4997, 1), (4997, 1))

In [11]:

#train test the data for modle
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=12)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3997, 1), (1000, 1), (3997, 1), (1000, 1))

# Bag of words

WORD2VEC

In [23]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
from gensim import models
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess

In [25]:
story = []
for word in df["review"]:
    raw_sent = sent_tokenize(word)
    for sent in raw_sent:
       story.append(simple_preprocess(sent))

In [27]:
df["review"][0]

"one    reviewers  mentioned   watching  1 oz episode   hooked.   right,    exactly  happened  me.  first thing  struck   oz   brutality  unflinching scenes  violence,  set  right   word go. trust me,     show   faint hearted  timid.  show pulls  punches  regards  drugs, sex  violence.   hardcore,   classic use   word.   called oz     nickname given   oswald maximum security state penitentary.  focuses mainly  emerald city,  experimental section   prison    cells  glass fronts  face inwards,  privacy   high   agenda. em city  home  many..aryans, muslims, gangstas, latinos, christians, italians, irish  more....so scuffles, death stares, dodgy dealings  shady agreements  never far away.  would say  main appeal   show  due   fact   goes   shows  dare. forget pretty pictures painted  mainstream audiences, forget charm, forget romance...oz  mess around.  first episode  ever saw struck    nasty   surreal,   say   ready  it,    watched more,  developed  taste  oz,  got accustomed   high level

In [26]:
story

[['one', 'reviewers', 'mentioned', 'watching', 'oz', 'episode', 'hooked'],
 ['right', 'exactly', 'happened', 'me'],
 ['first',
  'thing',
  'struck',
  'oz',
  'brutality',
  'unflinching',
  'scenes',
  'violence',
  'set',
  'right',
  'word',
  'go'],
 ['trust', 'me', 'show', 'faint', 'hearted', 'timid'],
 ['show', 'pulls', 'punches', 'regards', 'drugs', 'sex', 'violence'],
 ['hardcore', 'classic', 'use', 'word'],
 ['called',
  'oz',
  'nickname',
  'given',
  'oswald',
  'maximum',
  'security',
  'state',
  'penitentary'],
 ['focuses',
  'mainly',
  'emerald',
  'city',
  'experimental',
  'section',
  'prison',
  'cells',
  'glass',
  'fronts',
  'face',
  'inwards',
  'privacy',
  'high',
  'agenda'],
 ['em',
  'city',
  'home',
  'many',
  'aryans',
  'muslims',
  'gangstas',
  'latinos',
  'christians',
  'italians',
  'irish',
  'more',
  'so',
  'scuffles',
  'death',
  'stares',
  'dodgy',
  'dealings',
  'shady',
  'agreements',
  'never',
  'far',
  'away'],
 ['would',
  

In [29]:
model = Word2Vec(window=10,min_count=2)
model.build_vocab(story)
model.corpus_count

60574

In [30]:
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

(2898606, 3091735)

In [32]:
len(model.wv.index_to_key)

22988

In [34]:
def Documnt(doc):
  doc =[word for word in doc.split() if word in model.wv.index_to_key]
  return np.mean(model.wv[doc],axis=0)


In [38]:
Documnt(df["review"].values[0]).shape

(100,)

In [40]:
from tqdm import tqdm
x = []
for doc in tqdm(df["review"].values):
  x.append(Documnt(doc))

100%|██████████| 4997/4997 [02:12<00:00, 37.73it/s]


In [41]:
x=np.array(x)

In [44]:
y

,sentiment
0,1
1,1
2,1
3,0
4,1
...,...
4995,0
4996,1
4997,1
4998,0


In [45]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=12)

In [47]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.metrics import accuracy_score

model = GaussianNB()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.615

In [48]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.704

# Practical Advise
# 1: Ensemble technique use karni chahiye.
# 2: Huristic feature Use karny chahiye.
# 3: Deep learning is useful but not always.
# 4: if Deep learning than model will improve butt that is not sufficent.
# 5: Try to implement with balance datasets.
# 6: Try to solve project than you would handle project.